# Fx Forward Pricing

This notebook will run through the following business use cases :
* [Pricing a set of Fx Forwards using a the internal LUSID "Discounting" model](#pricing_bond)
* [Valuing a portfolio made up of an Fx forward position.](#pricing_bond_portfolio)
* [Valuing our Fx forward portfolios using an externally supplied Fx forward price.](#accrual_override)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of an FX forward instrument based on user provided parameters.](#bond_definition)
* [Supplying both simple and complex market data structures through the Quotes and the StructureMarketData store.](#structured_market_data)
* [Configuring recipes to value our FX forward in LUSID making use of the structured market data (GBP and USD OIS Yield Curves).](#recipe_configuration)
* [Running inline aggregations that value a unitised FX forward.](#accrued_interest)
* Running portfolio aggregations that value a porfolio containing FX forwards.
* [Using an externally valued FX forward price to calculate portd.](#external_bond_price)

<br>

For this notebook example we'll work the following set of GBP/USD Forwards with 3Y, 5Y and 10Y maturities:

* Trade Date : 02 Aug 2020
* Maturity Dates :
    * 02 Aug 2023 (3Y)
    * 02 Aug 2025 (5Y)
    * 02 Aug 2030 (10Y)
* Spot Rate : 1.2508
* Domestic Amount : 100,000,000
* Foreign Amounts :
    * 125,710,000 (3Y)
    * 126,485,000 (5Y)
    * 130,430,000 (10Y)


## Setup LUSID and LUSID API objects.

In [39]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from IPython.core.display import display
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)
from lusidtools.jupyter_tools.stop_execution import StopExecution


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)

# Setup the scope we'll use in this notebook:
scope = "fx-forward-pricing-nb"

LUSID Environment Initialised
LUSID SDK Version:  0.6.4748.0


In [40]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    df = pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])
    df['Effective At'] = df['Effective At'].dt.strftime('%d %b %Y')
    return df


## Define our Fx Forward Instruments

We'll start by defining our forwards using LUSID's internal representation of an Fx forward instrument. For a detailed
breakdown the internal representation take a look at the "FxForwardInstrument" section of the [LUSID API Swagger Specification](https://www.lusid.com/api/swagger/index.html)

### Initialise FxForward Parameters

Let's setup the basic parameters required for our set of Fx forwards:

In [41]:
trade_date = datetime(2020, 6, 1, tzinfo=pytz.utc)
#trade_date = datetime.today().replace(tzinfo=pytz.utc)
maturity_dates = {
    '1W' : datetime(2020, 6, 8, tzinfo=pytz.utc),
    '3Y' : datetime(2023, 6, 1, tzinfo=pytz.utc),
    '5Y' : datetime(2025, 6, 1, tzinfo=pytz.utc),
    '10Y': datetime(2030, 6, 1, tzinfo=pytz.utc)
}
spot_rate =  1.2508
dom_amount = 100000000
fgn_amounts = {
    '1W' : 125090000,
    '3Y' : 125710000,
    '5Y' : 126485000,
    '10Y': 130430000
}

### Create FxForwardInstrument Definitions

We can use those parameters to construct a LUSID representation of an FxForward. In our case we'll define a set made up of a
3Y, 5Y and 10Y GBP/USD Fx Forward.

In [42]:
def create_fx_forward_instrument_definition(dom_amount, fgn_amount, spot_rate, start_date, maturity_date):
    return models.FxForwardInstrument(
            dom_amount=dom_amount,
            fgn_amount=-fgn_amount,
            fgn_ccy="USD",
            ref_spot_rate=spot_rate,
            start_date=start_date.isoformat(),
            maturity_date=maturity_date,
            dom_ccy="GBP",
            instrument_type="FxForward")

gbpusd_fx_fwd_1w_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['1W'], spot_rate, trade_date, maturity_dates['1W'])
gbpusd_fx_fwd_3y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['3Y'], spot_rate, trade_date, maturity_dates['3Y'])
gbpusd_fx_fwd_5y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['5Y'], spot_rate, trade_date, maturity_dates['5Y'])
gbpusd_fx_fwd_10y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['10Y'], spot_rate, trade_date, maturity_dates['10Y'])

## Supply Market Data

Before we can price our Fx forwards we need to ensure LUSID has the required FX and interest rates available. This allows us
to showcase two ways of getting the market data we need to price an FX forward into LUSID. The simpler method for the FX rates
can simply be upserted as quotes but the more core complex structure of the FGPB and USD yield curves required the use of the
Structured Market Data store.

Let's start by upserting the GBP/USD FX rates for the trade data and a few subsequent dates we'll use to price the FX Forwards.
So we can see some movement in the forward prices we'll mock a drop in the spot rate on June 2nd followed by a sharp rise on June 3rd. When providing market
data to LUSID we also need to identify the supplier of the data and it's scope. The supplier is required further in the
notebook when we instruct LUSID how to source data for a particular aggregation.

### Upsert Spot Fx Rates

In [43]:
market_data_scope = 'fx-forward-pricing-nb-market-data'
market_supplier = 'Lusid'

trade_date_plus_one = datetime(2020, 6, 2, tzinfo=pytz.utc)
trade_date_plus_two = datetime(2020, 6, 3, tzinfo=pytz.utc)


def upsert_gbp_usd_fx_rate(rate, effective_at):
    upsert_quote_request = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="GBP/USD",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    # we need to insert the USD/GBP quote as well for use later in the notebook
    # when we're booking the sell leg of our forward as a transaction
    upsert_quote_request_inverse = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="USD/GBP",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=1/rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    response_gbp_usd = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"gbp-usd-01": upsert_quote_request})

    response_usd_gbp = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"usd-gbp-01": upsert_quote_request_inverse})

    if response_gbp_usd.failed or response_usd_gbp.failed:
        raise StopExecution(f"Failed to upload currency pairs:{response_gbp_usd.failed} or {response_usd_gbp.failed}")

    display(f"GBP/USD @ {rate} for {effective_at} uploaded to quote store.")
    display(f"USD/GBP @ {1/rate} for {effective_at} uploaded to quote store.")


upsert_gbp_usd_fx_rate(spot_rate, trade_date)
# drop in fx rate
upsert_gbp_usd_fx_rate(spot_rate - 0.0080, trade_date_plus_one)
# increase in fx rate
upsert_gbp_usd_fx_rate(spot_rate + 0.0080, trade_date_plus_two)


'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2428 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.804634695848085 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2588 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7944073721004131 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

### Interest Rate Curves

To price the FX Forwards we also need to supply the relevant interest rate curves for our domestic and foreign currencies. LUSID supports
storage of complex market data structures through the [Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr")
store. When upserting the interest rate curves we use a StructuredMarketDataId to not onl uniquely identify the curve but also provide
the supporting information required by the aggregation engine to correctly resolve the curves when we run our FX forward valuation.

In [44]:
def upsert_interest_rate_curve(ir_curve_json, scope, effective_at, market_asset):

    # provide the structured data file source and it's document format
    structured_market_data = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ir_curve_json)


    # create a unique identifier for our OIS yield curves
    structured_id = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="CurveVendorABC", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)

    upsert_request = models.UpsertStructuredMarketDataRequest(market_data_id=structured_id,
                                                          market_data=structured_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertStructuredMarketData
    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={market_asset : upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload interest rates curve {response.failed}")

    print(f"{market_asset} interest rate curve uploaded into scope={scope}")


def load_ois_curve_json(ccy):
    with open(f"data/{ccy}OIS50.json", "r") as ir_curve_json:
        return ir_curve_json.read()

ois_curve_gbp_json = load_ois_curve_json('GBP')
upsert_interest_rate_curve(ois_curve_gbp_json, market_data_scope, trade_date, "GBP/GBPOIS")

ois_curve_usd_json = load_ois_curve_json('USD')
upsert_interest_rate_curve(ois_curve_usd_json, market_data_scope, trade_date, "USD/USDOIS")

GBP/GBPOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data
USD/USDOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data


## Defining our Fx Forward Valuation

With our Fx forward instruments now defined we can move onto valuing them via LUSID's aggregation engine. But before we
can execute the aggregation we need to configure a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used")
instructing LUSID on how to value our Fx forwards. Specifically we need to select the Fx Forward pricing model to use and identify
where the aggregation can resolve the market data the model needs.

### Selecting a Pricing Model

The pricing model we wish to use is passed in through defining a PricingContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters. For this notebook we'll use the LUSID supplied "Discounting" model:

In [45]:
def create_pricing_context():
    vendor_model_rule = models.VendorModelRule(
        supplier="Lusid",
        model_name="Discounting",
        instrument_type="FxForward",
        parameters="{}")

    return models.PricingContext(
        model_rules=[vendor_model_rule]
    )

pricing_context = create_pricing_context()

### Selecting the Market Data

We can instruct LUSID on where to resolve market data required for pricing our Fx forwards through defining a MarketContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters. Recall that when we upserted our quote we passed in a supplier and a scope. One of
the powerful features in LUSID is the ability to run valuations against market data from different suppliers, or even define market data retrieval rules
to fallback to suppliers should your primary supplier not have the required data.

However our example is trivial as we only require the spot rate we loaded in earlier. So we'll define a simple market context :

In [46]:
def create_market_context():
    return models.MarketContext(
        # set rules for where we should resolve our rates data. In our case the interest rate curves we inserted into
        # the structured market result data store.
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        # control default options for resolving market data. In our case simply default to the LUSID market_supplier
        # and market data scope we defined earlier.
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our FX Forward Recipe

Now that we've defined what pricing model to use and where to source the market data we can bring those instructions
together in a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used"):

In [47]:
def create_fx_forward_pricing_recipe():
    return models.ConfigurationRecipe(
            scope=scope,
            code="fx-forward-pricing-recipe",
            description="Price Fx Forwards using LUSID internal model",
            market=market_context,
            pricing=pricing_context
        )

fx_forward_pricing_recipe = create_fx_forward_pricing_recipe()

## Price the FX Forwards

Let's summarise our current state:
 * We've defined a set of GBP/USD FX forwards for maturity in 3Y, 5Y and 10Y.
 * We've loaded in the spot GBP/USD rates for the dates we're going to value our forwards against.
 * We've setup our Recipe that configures how we would like to price our bond and where to source our required market data.

As we have no existing positions in the FX forward booked against a portfolio we'll run our aggregation on an inlined portfolio.
That is a portfolio made up of a set of weighted instruments only. In our case each request will be made up of only one instrument which
will be the FX forward we're valuing.

### Run an Aggregation to Price our Forwards

In [48]:
def run_fx_forward_pricing_aggregation(maturity, effective_at, fx_forward_instrument_definition, pricing_recipe):
    # setup weighted instrument (only our gilt definition)
    fx_forward = models.WeightedInstrument(quantity=1, instrument=fx_forward_instrument_definition, holding_identifier=f"{maturity}-holding")

    # create our aggregation request made up of our recipe and the metrics we would like to calculate (Bond PV)
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at,
        inline_recipe=pricing_recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    # As we're running an inline aggregation we must wrap our original aggregation request with an inline aggregation
    # request and pass in our weighted instruments
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[fx_forward]
    )

    # https://www.lusid.com/docs/api#operation/GetAggregationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)


Let's start by running the 5Y forward valuation across the three dates for which we upserted spot GBP/USD quotes. Recall the quotes were 1.2508 for
trade date, whic hthen dropped by 0.0080 on day two and subsequently then increased by 0.0160 on day 3.

In [49]:
result_t = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_one, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_two, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 5Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 5Y Forward PV,01 Jun 2020,-0.006160
1,GBP/USD 5Y Forward PV,02 Jun 2020,-0.012605
2,GBP/USD 5Y Forward PV,03 Jun 2020,0.000202


Let's now take a slightly different perspective and look at the prices across the Forward curve for one date only:

In [50]:
result_3y = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_5y = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_10y = run_fx_forward_pricing_aggregation('10Y', trade_date, gbpusd_fx_fwd_10y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_3y.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date, result_5y.data[0]['Holding/default/PV']],
    ['GBP/USD 10Y Forward PV', trade_date, result_10y.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.002024
1,GBP/USD 5Y Forward PV,01 Jun 2020,-0.006160
2,GBP/USD 10Y Forward PV,01 Jun 2020,-0.032075


## Pricing a Portfolio of FX Forwards

Up until now we've only priced a unit Fx Forward as an instrument. Let's now take it a step further and build a simple portfolio made up of
an FX forward positions. While we'll now be running our aggregation on an actual portfolio as opposed to just the Fx Forward instrument
we'll still make use of the same recipe we defined. This is because the rules regarding what model we'd like to use to price our
Fx Forward and where LUSID should retrieve market data from have not changed.


### Setting up our Portfolio

We'll firstly setup the portfolio that will hold our FX forward positions:

In [51]:
portfolio = "simple-fxfwd-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    succ, failed = format_portfolios_response(result)

    if not failed.empty:
        raise StopExecution(failed)

    return succ

create_portfolio(scope, portfolio, portfolio, "GBP")



,successful items
0,simple-fxfwd-portfolio-01


### Upserting our FX Forward Instrument

Previously as we were only pricing the FX Forward instrument we were only required to pass in it's instrument definition. As we're now
booking transactions against the instrument we'll need to upsert the instrument into LUSID. This will provide us with an identifier
we can use to book our transactions against.

For the purpose of our portfolio we'll only work with the 3Y FX forward from now on:

In [52]:
def upsert_fx_fwd_instrument(instrument_id, instrument_name, fx_fwd_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        # instrument display name
        name=instrument_name,
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        # our gilt instrument definition
        definition=fx_fwd_definition
    )}
    # Note we're using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition and not the base LUSID instruments.
    return instruments_api.upsert_lusid_instruments(bond_instrument_request)

gbpusd_fx_fwd_3y_instr_client_id = "gbp_usd_fwd_3y_12571"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_3y_instr_client_id, "GBP/USD 3Y @ 1.2571", gbpusd_fx_fwd_3y_instr_def)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gbpusd_fx_fwd_3y_instr_luid = instrument_creation_response.values[gbpusd_fx_fwd_3y_instr_client_id].lusid_instrument_id

gbpusd_fx_fwd_1w_instr_client_id = "gbp_usd_fwd_1w_12509"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_1w_instr_client_id, "GBP/USD 1W @ 1.2509", gbpusd_fx_fwd_1w_instr_def)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gbpusd_fx_fwd_1w_instr_luid = instrument_creation_response.values[gbpusd_fx_fwd_1w_instr_client_id].lusid_instrument_id


### Setting up our FX Forward Transaction

Let's assume we're taking a position in the GBP/USD 3Y Fx Forward we setup. As the domestic and foreign notionals are
included in the instrument definition we simply need to setup a transaction type that gives use a "stock" holding in the instrument. Now, as
there is no cash exchanged the default "Buy" transaction is unsuitable so we'll have to create our own. For details
on movements take a look at the notebook [Generating holdings with the movements engine in LUSID](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/ibor/Generating%20holdings%20with%20the%20movements%20engine%20in%20LUSID.ipynb)


#### Creating a Transaction Type

For more information on transaction types have a read of [Configuring transaction types](https://support.finbourne.com/configuring-transaction-types) and try the
 TODO find notebook

We need a transaction that simply buys a unit of the forward with no cash movement that we'll call "FxFwdInstBuy"

In [53]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstBuy",
        description="Buy a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="LongLonger",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstBuy_CM",
        description="Buy a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashFxForward",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

#### Upserting our transaction

With out transaction type configured we can now upsert a transation in the 3Y forward. An important point to note is when valuing
the Fx forward the calculated PV is a unitised value. Therefore our units in the transaction should match our notional:

In [54]:
def upsert_fwd_transaction(scope, txn_id, txn_type, txn_ccy, instrument_id, trade_date, settlement_date, portfolio, transaction_price, consideration):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type=txn_type,
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=settlement_date.isoformat(),
        units=100000000,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency=txn_ccy),
        exchange_rate=1,
        transaction_currency=txn_ccy
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])

# Insert position in 3Y

# our units are 100000000 as the PV in the forward pricing is calculated for a unit instrument.
units = 100000000
transaction_price = 1.2571  # spot + 3y fwd spread
consideration = 100000000
# upsert the buy leg of the forward
#upsert_fwd_transaction(scope, "fx_fwd_3y_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_3y_instr_client_id, trade_date, maturity_dates['3Y'], portfolio, transaction_price, consideration)
#upsert_fwd_transaction(scope, "fx_fwd_3y_b_001", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_3y_instr_client_id, trade_date, maturity_dates['3Y'], portfolio, transaction_price, consideration)


# Insert position in 1W
units = 100000000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 100000000
# upsert the buy leg of the forward
#upsert_fwd_transaction(scope, "fx_fwd_1w_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)
upsert_fwd_transaction(scope, "fx_fwd_1w_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)
upsert_fwd_transaction(scope, "fx_fwd_1w_b_001_CM", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)



### Value Portfolio

With our portfolio now populated with a transaction in the 3Y forward we can

In [55]:
def run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
                                 op='Value'),
            # models.AggregateSpec(key='Analytic/default/ValuationDate',
            #                              op='Value'),
            # models.AggregateSpec(key='Analytic/default/DomCcy',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/FgnCcy',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/StartDate',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/MaturityDate',
            #                      op='Value'),
            # models.AggregateSpec(key='Portfolio/default/Name',
            #                      op='Value'),
            # models.AggregateSpec(key='Holding/default/Cost',
            #                      op='Value'),
            # models.AggregateSpec(key='Holding/default/Units',
            #                      op='Value'),
            #             models.AggregateSpec("Holding/default/Error", "Value")
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)


result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,Portfolio PV,01 Jun 2020,-6351.446653
1,Portfolio PV,02 Jun 2020,-650089.508009
2,Portfolio PV,03 Jun 2020,629204.370517


In [56]:
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.002024
1,GBP/USD 3Y Forward PV,02 Jun 2020,-0.008455
2,GBP/USD 3Y Forward PV,03 Jun 2020,0.004325


### Price across the life of the forward

In [57]:
# assume spot rate on expiry date is lower thant forward rate
upsert_gbp_usd_fx_rate(1.2509, maturity_dates['1W'])


result_t = run_fx_forward_pricing_aggregation('1W', trade_date, gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('1W', trade_date_plus_one, gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)
result_on_maturity = run_fx_forward_pricing_aggregation('1W', maturity_dates['1W'], gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 1W Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 1W Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 1W Forward PV', maturity_dates['1W'], result_on_maturity.data[0]['Holding/default/PV']]
])


'GBP/USD @ 1.2509 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7994244144216165 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,GBP/USD 1W Forward PV,01 Jun 2020,-0.000064
1,GBP/USD 1W Forward PV,02 Jun 2020,-0.006501
2,GBP/USD 1W Forward PV,08 Jun 2020,0.000016


## Forward Pricing Override with LUSID Static Model

Let's now assume that we've calculated priced our FX forwards externally to LUSID but would like the prices in LUSID to
value our portfolio. We can do so by changing the pricing model we use from "Discounting" model to a "SimpleStatic" model. For
the static model to retrieve the externally calculate Forward prices we need to firsty upsert those prices as market data quotes against the
FX forward insturment we defined earlier in the notebook

### Load FX Forward Prices into LUSID

In [58]:
def upsert_external_fx_forward_as_quote(fx_fwd_price, effective_at):
    spot_quote = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=gbpusd_fx_fwd_3y_instr_luid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Spread',
                field='Mid'),
            effective_at=effective_at,
        ),
        metric_value=models.MetricValue(
            value=fx_fwd_price,
            unit='GBP'),
        lineage='InternalSystem')

    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=market_data_scope,
        request_body={"1": spot_quote})

    if response.failed:
        raise StopExecution(f"Failed to upload FX Forward price:{response.failed}")

    print(f"FX Forward @{fx_fwd_price} uploaded into Quote store.")

upsert_external_fx_forward_as_quote(0.10000, trade_date)
upsert_external_fx_forward_as_quote(0.105000, trade_date_plus_one)
upsert_external_fx_forward_as_quote(0.005000, trade_date_plus_two)


FX Forward @0.1 uploaded into Quote store.
FX Forward @0.105 uploaded into Quote store.
FX Forward @0.005 uploaded into Quote store.


### Updating our Recipe to the Static Model

We now need to use a recipe that selects the Static model for pricing forwards. As we did earlier in the notebook we select
the model we'd like to use through the PricingContext of the recipe

In [59]:
def create_static_fx_forward_pricing_context():
    return models.PricingContext(
        # the default behaviour does not allow looking up data for pricing instruments so we must allow it.
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="FxForward",
                parameters="{}"
            )
        ]
    )

static_fx_forward_pricing_context = create_static_fx_forward_pricing_context()


### Instruct LUSID where to find the forward price

Additionally we need to make the LUSID aggregation aware of where to resolve the FX forward prices we've inserted.

In [60]:
def create_static_fx_forward_pricing_market_context():
    return models.MarketContext(
        market_rules=[
            # additional rule to resolve our quote
            models.MarketDataKeyRule(
                key='Equity.LusidInstrumentId.*',
                supplier=market_supplier,
                data_scope=market_data_scope,
                quote_type='Spread',
                field='Mid'),
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope,
            manifest_level_of_detail="Full")
    )

    return market_context

static_fx_forward_pricing_market_context = create_static_fx_forward_pricing_market_context()

### Construct Static Model Fx Forward Recipe

Let's create a new recipe based on the pricing and market contexts we've just created

In [61]:
def create_static_fx_forward_pricing_recipe(scope, market_context, pricing_context):

    return models.ConfigurationRecipe(
        scope=scope,
        code="static-fx-forward",
        description="Price forward using prices from the quote store.",
        market=market_context,
        pricing=pricing_context
    )

static_fx_forward_price_config_recipe = create_static_fx_forward_pricing_recipe(scope, static_fx_forward_pricing_market_context, static_fx_forward_pricing_context)


### Rerun Pricing with Static Model and compare with the Discounting Model

We can now price our Fx Fwd portfolio using the Static model (which uses our external FX forward price we fed in as a quote)
and compare the valuation with a corresponding aggregatoin but using the discounting model

In [62]:
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

manifest_df = pd.DataFrame()
for manifest in result_t.manifests:
    for manifest_quote in manifest.quotes:
        quote_df = lusid_response_to_data_frame([manifest_quote.quote])
        quote_df.insert(0, "market_data_rule", manifest_quote.rule)
        manifest_df = manifest_df.append(quote_df)

display(manifest_df)


,market_data_rule,quote_id.quote_series_id.provider,quote_id.quote_series_id.instrument_id,quote_id.quote_series_id.instrument_id_type,quote_id.quote_series_id.quote_type,quote_id.quote_series_id.field,quote_id.effective_at,metric_value.value,metric_value.unit,lineage,cut_label,uploaded_by,as_at
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/USD,CurrencyPair,Price,mid,2020-06-01T00:00:00.0000000+00:00,1.250800,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:49.320601+00:00


In [63]:
upsert_external_fx_forward_as_quote(0.50000, trade_date)
upsert_external_fx_forward_as_quote(0.505000, trade_date_plus_one)
upsert_external_fx_forward_as_quote(0.605000, trade_date_plus_two)

#spot_rate = 1.2508
spot_rate = 1.5
upsert_gbp_usd_fx_rate(spot_rate, trade_date)
# drop in fx rate
upsert_gbp_usd_fx_rate(spot_rate - 0.0080, trade_date_plus_one)
# increase in fx rate
upsert_gbp_usd_fx_rate(spot_rate + 0.0080, trade_date_plus_two)

FX Forward @0.5 uploaded into Quote store.
FX Forward @0.505 uploaded into Quote store.
FX Forward @0.605 uploaded into Quote store.


'GBP/USD @ 1.5 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.6666666666666666 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.492 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.6702412868632708 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.508 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.6631299734748011 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

In [64]:
result_static_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date)
result_static_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_one)
result_static_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_two)

In [65]:
manifest_df = pd.DataFrame()
for manifest in result_static_t.manifests:
    for manifest_quote in manifest.quotes:
        quote_df = lusid_response_to_data_frame([manifest_quote.quote])
        quote_df.insert(0, "market_data_rule", manifest_quote.rule)
        manifest_df = manifest_df.append(quote_df)

display(manifest_df)



,market_data_rule,quote_id.quote_series_id.provider,quote_id.quote_series_id.instrument_id,quote_id.quote_series_id.instrument_id_type,quote_id.quote_series_id.quote_type,quote_id.quote_series_id.field,quote_id.effective_at,metric_value.value,metric_value.unit,lineage,cut_label,uploaded_by,as_at
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/USD,CurrencyPair,Price,mid,2020-06-01T00:00:00.0000000+00:00,1.500000,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.195659+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,USD/GBP,CurrencyPair,Price,mid,2020-06-01T00:00:00.0000000+00:00,0.666667,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.283069+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/GBP,CurrencyPair,Price,mid,2020-07-07T12:25:57.9767915+00:00,1.000000,GBP,,,lusid-same-currency,2020-07-07 12:25:57.981698+00:00


In [66]:
manifest_df = pd.DataFrame()
for manifest in result_static_t_plus_one.manifests:
    for manifest_quote in manifest.quotes:
        quote_df = lusid_response_to_data_frame([manifest_quote.quote])
        quote_df.insert(0, "market_data_rule", manifest_quote.rule)
        manifest_df = manifest_df.append(quote_df)

display(manifest_df)


,market_data_rule,quote_id.quote_series_id.provider,quote_id.quote_series_id.instrument_id,quote_id.quote_series_id.instrument_id_type,quote_id.quote_series_id.quote_type,quote_id.quote_series_id.field,quote_id.effective_at,metric_value.value,metric_value.unit,lineage,cut_label,uploaded_by,as_at
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/USD,CurrencyPair,Price,mid,2020-06-02T00:00:00.0000000+00:00,1.492000,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.382520+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,USD/GBP,CurrencyPair,Price,mid,2020-06-02T00:00:00.0000000+00:00,0.670241,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.471599+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/GBP,CurrencyPair,Price,mid,2020-07-07T12:25:58.2914497+00:00,1.000000,GBP,,,lusid-same-currency,2020-07-07 12:25:58.302022+00:00


In [67]:
manifest_df = pd.DataFrame()
for manifest in result_static_t_plus_two.manifests:
    for manifest_quote in manifest.quotes:
        quote_df = lusid_response_to_data_frame([manifest_quote.quote])
        quote_df.insert(0, "market_data_rule", manifest_quote.rule)
        manifest_df = manifest_df.append(quote_df)

display(manifest_df)


,market_data_rule,quote_id.quote_series_id.provider,quote_id.quote_series_id.instrument_id,quote_id.quote_series_id.instrument_id_type,quote_id.quote_series_id.quote_type,quote_id.quote_series_id.field,quote_id.effective_at,metric_value.value,metric_value.unit,lineage,cut_label,uploaded_by,as_at
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/USD,CurrencyPair,Price,mid,2020-06-03T00:00:00.0000000+00:00,1.508000,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.560356+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,USD/GBP,CurrencyPair,Price,mid,2020-06-03T00:00:00.0000000+00:00,0.663130,rate,FxDataVendorABC,,00u78w98jnujIujIc2p7,2020-07-07 12:25:57.641470+00:00
0,"{'as_at': None,\n 'data_scope': 'fx-forward-pr...",Lusid,GBP/GBP,CurrencyPair,Price,mid,2020-07-07T12:25:58.5201441+00:00,1.000000,GBP,,,lusid-same-currency,2020-07-07 12:25:58.523924+00:00


In [68]:
aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 3Y Forward PV (Static),01 Jun 2020,16606666.666667
1,GBP/USD 3Y Forward PV (Static),02 Jun 2020,16159517.426273
2,GBP/USD 3Y Forward PV (Static),03 Jun 2020,17049071.618037


In [69]:
result_discounting_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_discounting_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_discounting_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date, result_discounting_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_one, result_discounting_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_two, result_discounting_t_plus_two.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV (Discounting),01 Jun 2020,16607764.014449
1,GBP/USD 3Y Forward PV (Static),01 Jun 2020,16606666.666667
2,GBP/USD 3Y Forward PV (Discounting),02 Jun 2020,16160629.470908
3,GBP/USD 3Y Forward PV (Static),02 Jun 2020,16159517.426273
4,GBP/USD 3Y Forward PV (Discounting),03 Jun 2020,17050154.424903
5,GBP/USD 3Y Forward PV (Static),03 Jun 2020,17049071.618037


### Update our Static Forward Rates To Match Discounting Model

Let's finally try to match the forward rates of the static and idiscounting model by using the discounting model to
calculate numbers and the feeding into quote store

In [70]:
# price the forward sintruments
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)

# upsert the forward prices of the instrument from discounting model into our quote store

upsert_external_fx_forward_as_quote(result_t.data[0]['Holding/default/PV'], trade_date)
upsert_external_fx_forward_as_quote(result_t_plus_one.data[0]['Holding/default/PV'], trade_date_plus_one)
upsert_external_fx_forward_as_quote(result_t_plus_two.data[0]['Holding/default/PV'], trade_date_plus_two)

# run pricing of our entire portfolios now using both static and discounting models

result_static_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date)
result_static_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_one)
result_static_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

result_discounting_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_discounting_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_discounting_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date, result_discounting_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_one, result_discounting_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_two, result_discounting_t_plus_two.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

FX Forward @0.1639485155659402 uploaded into Quote store.
FX Forward @0.15948171082083304 uploaded into Quote store.
FX Forward @0.1683679271572531 uploaded into Quote store.


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV (Discounting),01 Jun 2020,16607764.014449
1,GBP/USD 3Y Forward PV (Static),01 Jun 2020,16606666.666667
2,GBP/USD 3Y Forward PV (Discounting),02 Jun 2020,16160629.470908
3,GBP/USD 3Y Forward PV (Static),02 Jun 2020,16159517.426273
4,GBP/USD 3Y Forward PV (Discounting),03 Jun 2020,17050154.424903
5,GBP/USD 3Y Forward PV (Static),03 Jun 2020,17049071.618037
